# Playing with Linear Regression

Using the same example from Pierre, same data, but we can try to do this using basic machine learning.
https://github.com/axiomiety/crashburn/blob/master/jupyter/simple_linear_regression.ipynb

### Import the same datasets (and other setup libraries/functions)

In [1]:
import requests, pandas, io
import time, itertools
import myutils

url='http://www.stat.ufl.edu/~winner/data/brainhead.dat'
data=requests.get(url)
col_names=('gender', 'age_range', 'head_size', 'brain_weight')
col_widths=[(8,8),(16,16),(21-24),(29-32)]
df=pandas.read_fwf(io.StringIO(data.text), names=col_names, colspec=col_widths)
df.head()

dfs = [df, 
       churn(df,4),
       churn(df,8),
       churn(df,12),
       churn(df,16)]

SyntaxError: invalid syntax (myutils.py, line 182)

First before I start I want to do some sizing tests..  like a few magnitudes more data,
so we make an array of dataFrames dfs[]

In [18]:
print('dfs[0-4] #rows:')
for d in dfs:
    print (d.shape[0])


dfs[0-4] #rows:
237
3792
60672
970752
15532032


Ok I think these 5 samples is enough.

## Analytical Method (and Performance) 


In [53]:
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.stats import linregress

timings = {'rows':[],'time':[]}
#timings.append(['data','time'])
for d in dfs:
    r = time_fn(linregress,d.head_size,d.brain_weight)
    timings['rows'].append(d.shape[0])
    timings['time'].append(r[1]*1000)
t = pandas.DataFrame(timings)
rt = t['rows']/t['time']
t['rt'] = rt
display(t)

for d in dfs:
    print(linregress(d.head_size,d.brain_weight))


,rows,time,rt
0,237,5.974,39.671912
1,3792,4.681,810.083316
2,60672,1.999,30351.175588
3,970752,62.855,15444.308329
4,15532032,1143.931,13577.769988


LinregressResult(slope=0.26342933948939939, intercept=325.57342104944235, rvalue=0.79956970925429616, pvalue=5.9576308394065412e-54, stderr=0.012907433440886988)
LinregressResult(slope=0.26342933948939917, intercept=325.57342104944314, rvalue=0.79956970925429593, pvalue=0.0, stderr=0.0032140617796317652)
LinregressResult(slope=0.26342933948939917, intercept=325.57342104944314, rvalue=0.79956970925429582, pvalue=0.0, stderr=0.00080331675985773965)
LinregressResult(slope=0.26342933948939984, intercept=325.57342104944075, rvalue=0.79956970925430071, pvalue=0.0, stderr=0.00020082608673381252)
LinregressResult(slope=0.26342933948942276, intercept=325.57342104935742, rvalue=0.79956970925446968, pvalue=0.0, stderr=5.0206473196643748e-05)


The perf table shows it isn't bad.  It is scaling better than I thought.  

Doesn't prove what everyone says, analytic solver good for < 10k or so size samples.  Even at 15m points it solves it fast.   (??)


### Performance Sidebar
For fun, wanted to compare performance w/ Pierre's solution (vs the scipy library)

In [9]:
def pierre_solver (df):
    brain_weight_sample_mean = df['brain_weight'].mean()
    head_size_sample_mean = df['head_size'].mean()
    b_1 = sum(df.apply(lambda r: (r['head_size']-head_size_sample_mean)*(r['brain_weight']-brain_weight_sample_mean), axis=1))
    b_1 /= sum(df.apply(lambda r: (r['head_size']-head_size_sample_mean)**2, axis=1))
    b_0 = brain_weight_sample_mean-b_1*head_size_sample_mean
    return [b_0, b_1]

# prove the solutions are about same:
print (pierre_solver(dfs[0]))
print (pierre_solver(dfs[1]))

# run first 2 
timings = {'rows':[],'time':[]}
for d in dfs[:3]:
    r = time_fn(pierre_solver,d)
    timings['rows'].append(d.shape[0])
    timings['time'].append(r[1]*1000)
t = pandas.DataFrame(timings)
rt = t['rows']/t['time']
t['rt'] = rt
display(t)



[325.57342104944223, 0.26342933948939945]
[325.57342104944132, 0.26342933948939967]


,rows,time,rt
0,237,17.438,13.591008
1,3792,195.380,19.408332
2,60672,3490.342,17.382824


Alot slower for obvious reasons (demo code, unoptimized etc).  It also shows as n grows > 1000's its infeasible to solve analytically... this crashes doing the 100k+ iteration (or never finishes).

# Training Method 

Things needed for linear a regression line, of form f(x) = Ax + B (activation function)  
- Find optimal A, B values & cost function J(A,B)  
- Need partial derivatives for J(A,B) - dA and dB   
- Iterate A = A + u * dA && B = B + u * dB  
- Pretty much can start at random pt or 0,0 and do Gradient Descent  
- Step until close to 0 - define step size (u) and when to stop  

First a basic gradient descent example for a simple function f (x^2 - 2x + 1)
The derivative of this is 2x - 2, start at x=0, and solve where y=0

### Gradient Descent Dummy Sample (1)


In [1]:
from scipy.misc import derivative

def f(x):
    return x**2 - 2*x + 1

def p(f, x):
    return derivative(f,x)

def grad_descent(f):
    x = 0
    step = 0.1
    error = 0.5

    # loop while error > err_lmt
    while (error > 0.01):
        x = x - step*(p(f,x)) 
        print ('x=',x, 'y=',f(x))
        error = abs(f(x)-0)
    print('error: ',error)
    print('x: ',x)

grad_descent(f)


x= 0.2 y= 0.64
x= 0.36 y= 0.4096
x= 0.488 y= 0.262144
x= 0.5904 y= 0.16777216
x= 0.67232 y= 0.1073741824
x= 0.737856 y= 0.068719476736
x= 0.7902848 y= 0.043980465111
x= 0.83222784 y= 0.0281474976711
x= 0.865782272 y= 0.0180143985095
x= 0.8926258176 y= 0.0115292150461
x= 0.91410065408 y= 0.00737869762948
error:  0.00737869762948
x:  0.91410065408


### Putting it together

Training set: x[n] ~ df['head_size'] 
Solution set: y[n] ~ df['brain_weight']
Hypothesis: h(x):  Ax + B  
Cost P(h[x]) = P(A,B) = 1/n * sum( h(x[n]) - y[n] )^2  

Consider P(A,B) = contour/3d of all costs of every combination A,B (0x+0, 1x+1, 0x+1, etc...)  
Optimal is where P(A,B) = minimum  

Start with guess Optimal P = A=1, B=1, h(x) = 1x+1  

Optimal P(A,B) =  
 * A = A - step * dP/dA  <- initial guess +/- partial derivative of A (ie, movement of A parameter)      
 * B = B - step * dP/dA  <- initial guess +/- partial derivative of B (ie, movement of B parameter)  
Repeat until close to step_limit  


In [29]:
import sympy as sp
from sympy.core.compatibility import as_int
import sympy.concrete.summations as sum
import itertools

# evaluate/calculate f with data sub for x and y (very slow iterative)
def evalSumF(f,x,y,testData):
    n=0
    for _,d in testData.iterrows():  # global test data
        n += f.subs(x,d.head_size).subs(y,d.brain_weight)
    return n * (1.0/len(testData))

# generate partial derivative of e, with respect to v, for testData (x,y) and evaluate
def evalPartialDeriv(e,x,y,testData,v,guessV,o,guessO):
    pc = evalSumF(sp.diff(e,v),x,y,testData)
    pceval = pc.subs(v,guessV).subs(o,guessO)
    return pceval

# hard coded solver, start w/ guess, solve cost, iterate cost+/-partialDerivs
def grad_descent2(testData):
    guessA = guessB = 1.0   #initial guess y=1x+1

    stepA = 0.00000005   #dif step for diff A,B ?
    stepB = 0.25         #maybe normalize data first
    step_limit = 0.0001  # when to stop, when cost stops changing
    loop_limit = 5       # arbitrary max limits
    costChange = 1.0

    A,B,x,y = sp.symbols('A B x y')
    f = A*x + B  # linear func y=mx+b
    e = (f - y)**2  # error squared
    print ('init guess A: %f, B: %f'%(guessA,guessB))
    print ('init func: %s, test size: %d' %(str(f),testData.shape[0]))
    
    costF = evalSumF(e,x,y,testData)  # cost fun evaluted for testData
    print('init costF',str(costF)[:80])
    costEval = costF.subs(A,guessA).subs(B,guessB)  # cost evaluted for A B guess
    print('init cost',costEval)

    i=0  
    while (abs(costChange) > step_limit and i<loop_limit):  # arbitrary limiter
        pda = evalPartialDeriv(e,x,y,testData,A,guessA,B,guessB)
        pdb = evalPartialDeriv(e,x,y,testData,B,guessB,A,guessA)
        guessA = guessA - stepA * pda
        guessB = guessB - stepB * pdb
        previousCost = costEval
        costEval = costF.subs(A,guessA).subs(B,guessB)
        costChange = previousCost-costEval
        print ('i=%d,cost=%d,A=%f,B=%f'%(i, int(costEval), guessA, guessB))
        i=i+1
    return guessA,guessB

timings = []
r = time_fn(grad_descent2,df)
print ('finished for rows,time(s)',d.shape[0], r[1])


init guess A: 1.000000, B: 1.000000
init func: A*x + B, test size: 237
init costF 0.00421940928270042*(2720.0*A + B - 955.0)**2 + 0.00421940928270042*(2773.0*A + 
init cost 5609738.70886076
i=0,cost=3871290,A=0.135457,B=-1175.059072
i=1,cost=2672945,A=0.851485,B=-192.217064
i=2,cost=1846896,A=0.255257,B=-1001.816037
i=3,cost=1277474,A=0.748530,B=-323.272384
i=4,cost=884947,A=0.337256,B=-880.275431
finished for rows,time(s) 15532032 14.868650000000002


Above shows it getting closer after 5 test iterations - cost is decreasing and A,B params are converging (?) towards solution of 0.26x + 325.   But this is super slow (32sec for 5 iterations)... and guess what, I ran this to near completion and it did converge after 1800 iterations!!  


Other observations  
- the initial A,B didn't matter as much as you'd think  
- the step sizes for A,B really matter - wrong steps are you step widely over the solution back and forth.  Given A target range for a slope is small, and Y intercept is large, maybe we need to normalize/scale first?   Random fine tuning steps was required.  
- it is super slow, in theory you can find a solution for large testData sets this way where analytic solutions wont work.  But my lame code using sympy is so slow.  Have to find a better library or more to matrix multiplication (not sure if I'll rewrite (gradient_descent3) using matrices but I should).  

Visually showing the solution forming is interesting

### Optimizations

Gradient Descent (aka Batch Gradient Descent), while in theory faster than solving via the the normal equation (examples to come later), or the builtin solvers when the # of features or size of training set is very large, is still very slow overall.  *In Batch GD, every step takes the cost down and steps towards the final solution, since solving the derivatives for all test data always points you in the right direction*
 
There are 2 optimizations to the GD algo above  

1.  Stochastic GD   
    + Step A - Add a shuffle to the test data set
    + Step B - instead of solving the partial derivative for all test data, solve for just 1 and take a step  
    + *Solving derivatives for just random testData doesn't always take your cost down, but testing and logic shows it   jumps around a little but get you there quickly*  
    + *Each step is way faster like 1/n times faster where n is size of test data*  


2.  Mini-Batch GD  
    * Step A - Add a shuffle to the test data set
    * Step B - instead of solving the partial derivative for all test data, solve for a small batch like 10 and take a  step  
    * *Solving a small batch makes it more likely to step in the right directin (cost down), and testing agrees*
    * *Each step is also way faster like batchSize/n times faster* 

Sample results of a test run of 1,000 iterations of standard B-GD, S-GD, and MB-GD (size 10) for test data size of 200


<table>
    <tr><td>*</td><td>B-GD</td><td>MB-GD</td><td>S-GD</td></tr>
    <tr><td>A</td><td>0.28</td><td>0.27</td><td>0.19</td></tr>
    <tr><td>B</td><td>253</td><td>365</td><td>477</td></tr>
    <tr><td>Timing(s)</td><td>1899</td><td>702</td><td>582</td></tr>
</table>

(Reminder target solutions are A = 0.26, B=325)

I should graph the results because its not evident from above that the results of MB-GD and S-GD actually jump around close  to the solution for some time but do not converge very well, while B-GD always moves slowly towards the final answer.   Some fine tuning required I think, as MB-GD and S-GD don't need 1,000 iterations.




## Normal Equation - Linear Algebra

Final note, the Normal Equation can be used which is really quite easy as shown below using numpy to calculate

![Normal Equation Image](https://csharpcorner-mindcrackerinc.netdna-ssl.com/article/normal-equation-implementation-from-scratch-in-python/Images/image001.png)



In [4]:
import numpy as np  

X = np.matrix('1,1;1,2;1,4;1,5')     # make sample matrix of test X head size data, note u have to put a dummy 1 column
Y = np.matrix('10;20;40;50')         # make sample matrix of correlated Y(X) brain weight data

(X.T.dot(X)).I.dot(X.T).dot(Y)       # We can expect a simple line of Y=10x+0

matrix([[ -2.48689958e-14],
        [  1.00000000e+01]])

To make it more realistic, we'll solve for the real data set as well using the original examples of data from http://www.stat.ufl.edu/~winner/data/brainhead.dat


In [50]:
df=pandas.read_fwf(io.StringIO(data.text), names=col_names, colspec=col_widths)

def normal_solver(d):
    # some setup to make a ndarray
    X = pandas.DataFrame({'Bias' : 1,'head_size' : d['head_size']})
    Y = d['brain_weight']
    X = np.asmatrix(X.as_matrix())
    Y = Y.as_matrix()
    return (X.T.dot(X)).I.dot(X.T).dot(Y)

normal_solver(df)

matrix([[  3.25573421e+02,   2.63429339e-01]])

Results look good, Y = 0.26x + 325 as expected !

For kicks I timed the normal equation... and interesting.. linregress is way faster, scales better (not surprised)


In [65]:
timings = {'rows':[],'normaltime':[],'linegresstime':[]}
for d in dfs:
    r  = time_fn(normal_solver,d)
    r2 = time_fn(linregress,d.head_size,d.brain_weight)
    timings['rows'].append(d.shape[0])
    timings['normaltime'].append(r[1]*1000)
    timings['linegresstime'].append(r2[1]*1000)
t = pandas.DataFrame(timings)
rt = t['rows']/t['normaltime']
t['rateA'] = rt
rt2 = t['rows']/t['linegresstime']
t['rateB'] = rt2
display(t)

,linegresstime,normaltime,rows,rateA,rateB
0,1.016,7.257,237,32.658123,233.267717
1,0.629,17.338,3792,218.710347,6028.616852
2,20.521,26.084,60672,2326.023616,2956.581063
3,77.664,119.651,970752,8113.195878,12499.381953
4,1280.261,1676.283,15532032,9265.757632,12131.926224
